# **Bibliotecas**

In [55]:
import pandas as pd
from bs4 import BeautifulSoup
import requests as rq
import re

# **Extração dos Dados**

In [4]:
def armazenaLivros(total_paginas):

  dados_livros = []

  for pagina in range(1, total_paginas + 1):

    url = f'https://books.toscrape.com/catalogue/page-{pagina}.html'
    resposta = rq.get(url)
    soup = BeautifulSoup(resposta.text, 'html.parser')

    livros = soup.find_all('h3')

    for livro in livros:
      url_livro = livro.find('a')['href']
      resposta_livro = rq.get('https://books.toscrape.com/catalogue/' + url_livro)
      soup_livro = BeautifulSoup(resposta_livro .text, 'html.parser')

      titulo = soup_livro.find('h1').text
      categoria = soup_livro.find('ul', class_='breadcrumb').find_all('a')[2].text.strip()

      #Preço
      preco = soup_livro.find('p', class_='price_color').text.strip()
      match = re.search(r'\d+\.\d+', preco)
      if match:
        preco = float(match.group())

      # Avaliação
      avaliacao = soup_livro.find('p', class_='star-rating')['class'][1]
      dict_avaliacoes = {
          'One': 1,
          'Two': 2,
          'Three': 3,
          'Four': 4,
          'Five': 5
        }
      avaliacao = dict_avaliacoes.get(avaliacao)

      #Em Estoque
      disponibilidade = soup_livro.find('p', class_='instock availability').text.strip()
      qtd_em_estoque = disponibilidade.split('In stock')
      qtd_em_estoque = qtd_em_estoque[1].strip()

      # Extraindo apenas a quantidade em estoque:
      em_estoque = None
      match = re.search(r'\d+', qtd_em_estoque)
      if match:
        em_estoque = int(match.group())

      dados_livros.append([titulo, categoria, avaliacao, preco, em_estoque])

  return dados_livros

In [5]:
total_paginas = 50
lista_livros = armazenaLivros(total_paginas)

In [52]:
len(lista_livros)

1000

In [53]:
df = pd.DataFrame(data=lista_livros, columns=['titulo','categoria','avaliacao','preco','em_estoque'])

In [54]:
df.head()

,titulo,categoria,avaliacao,preco,em_estoque
0,A Light in the Attic,Poetry,3,51.77,22
1,Tipping the Velvet,Historical Fiction,1,53.74,20
2,Soumission,Fiction,1,50.10,20
3,Sharp Objects,Mystery,4,47.82,20
4,Sapiens: A Brief History of Humankind,History,5,54.23,20
